In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!curl -O https://raw.githubusercontent.com/DUNE/larnd-sim/refs/heads/develop/larndsim/bin/response_44_v2a_full.npz

In [ ]:
def gauss(x, mu, sigma):
    return 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2/sigma/sigma/2)

In [ ]:
def field_response():
    response = np.load('response_44_v2a_full.npz')
    return response

In [ ]:
q = gauss(np.arange(-10, 10, 0.1), 0, 0.3)
qunit = gauss(np.arange(-10, 10, 0.1), 0, 0.3)

q *= 2.4
# q = np.roll(q, 40)
plt.plot(q)

In [ ]:
fr0 = field_response()['response'][0,0]
fr0 = fr0.reshape(-1, 2).sum(axis=-1)[-1801:]
fr0 *= 0.05

In [ ]:
print(np.sum(fr0))

In [ ]:
wf = np.convolve(q, fr0)

In [ ]:
plt.plot(wf)

In [ ]:
def deconv(wf, k):
    deconvq = np.fft.ifft(np.fft.fft(wf, n=len(wf)) / np.fft.fft(k, n=len(wf))).real
    return deconvq[:len(wf)+1-len(k)]

In [ ]:
deconvq = deconv(wf, fr0)

In [ ]:
plt.plot(deconvq[:200], '--', label='qhat')
plt.plot(q, '.-', label='true')
plt.legend()

In [ ]:
def trigger(wf, thres):
    return np.argmax(np.cumsum(wf)>thres)
# integrate every k ticks
def integrate_k(wf, k):
    if wf.shape[0] % k:
        # raise ValueError('wf.shape[0] % k != 0')
        wf = np.pad(wf, (0, k - wf.shape[0] % k))
        print('warning: length of wf is now', len(wf))
    return wf.reshape(-1, k).sum(axis=-1)
def trigger_integrate_k(wf, k, start_idx):
    if wf[start_idx:].shape[0] % k:
        # raise ValueError('wf.shape[0] % k != 0')
        wf = np.pad(wf[start_idx:], (0, k - wf[start_idx:].shape[0] % k))
        print('warning: length of wf is now', len(wf))
    else:
        wf = wf[start_idx:]
    return wf.reshape(-1, k).sum(axis=-1), start_idx

In [ ]:
kticks = 30
thres = 5
wf_k_trunc, start_idx_fg = trigger_integrate_k(wf, kticks, trigger(wf, thres))
fr0_k = integrate_k(fr0, kticks)
plt.plot(start_idx_fg + np.arange(0, len(wf_k_trunc)) * kticks, wf_k_trunc, 'o-')
plt.xlabel('time tick[50ns]')
plt.ylabel(f'recorded q in {kticks} ticks')
print(wf_k_trunc)

In [ ]:
def lost_waveform(wf_trunc, kticks, fr_fg, thres, start_idx_fg):
    ''' assume fr is full length without downsampling'''
    qmax = np.max(np.cumsum(wf_trunc)) + thres
    print('qmax', qmax)
    idx = np.argmin(np.abs(thres/qmax - np.cumsum(fr_fg)/np.sum(fr_fg)))
    print(idx, np.cumsum(fr_fg)[idx], thres/qmax)
    lost = np.zeros(start_idx_fg) # at threshold
    # interpolate accoriding to wf_trunc and fr[idx:] to lost waveform and fr[:idx]
    # wf_trunc[0] is cumulative from -inf, wf_trunc[1:] is cumulative per interval
    if idx < start_idx_fg:
        print('ok, good')
        lost[-idx:] = fr_fg[:idx] * thres / np.sum(fr_fg[:idx])
        print(np.sum(lost))
    else:
        print('not good')
        lost = fr_fg[idx-start_idx_fg:idx] * thres / fr_fg[idx]
    ncycles = len(lost) // kticks
    print(ncycles, len(wf_trunc))
    wf_full = np.zeros(ncycles + len(wf_trunc))
    wf_full[:ncycles] = lost[-ncycles * kticks:].reshape(ncycles, kticks).sum(axis=-1)
    wf_full[ncycles] = wf_trunc[0]
    print(wf_full[ncycles], wf_trunc[0], thres)
    wf_full[ncycles+1:] = wf_trunc[1:]
    return wf_full

In [ ]:
wf_full_k = lost_waveform(wf_k_trunc, kticks, fr0, thres, start_idx_fg)

In [ ]:
plt.plot(np.arange(len(wf_full_k))*kticks, wf_full_k/kticks, label=f'guess / {kticks}')
plt.plot(wf, label='true waveform')
plt.legend()
print(np.sum(wf_full_k), len(wf_full_k))
plt.xlabel('time tick')

In [ ]:
qdeconv2 = deconv(wf_full_k, fr0_k)
print(np.sum(wf_k_trunc), np.sum(wf_full_k), np.sum(np.convolve(wf_full_k, q/np.sum(q))), np.sum(fr0_k), 'qdeconv2', np.sum(qdeconv2), np.sum(wf), np.sum(wf[:start_idx_fg]))

print(qdeconv2.shape)
print(np.sum(qdeconv2), np.sum(q), np.sum(deconvq))
print(np.argmax(qdeconv2)*kticks, np.argmax(q), np.argmax(deconvq))
print(np.std(qdeconv2), np.std(q), np.std(deconvq))

print(len(qdeconv2), 'length')
plt.plot(np.arange(len(qdeconv2))*kticks, qdeconv2/kticks, label=f'Average over {kticks}')
# plt.plot(deconvq, label='qhat')
plt.plot(q, label='true')
plt.xlabel('Time tick[50ns]')
plt.xlabel('q')
plt.title('Charge')
plt.legend()

In [ ]:
plt.plot(np.arange(len(qdeconv2))*kticks, qdeconv2)

In [ ]:
wf_full = np.interp(np.arange(0, len(wf)), np.arange(0, len(wf_full_k)+1)*kticks, np.cumsum([thres,] + wf_full_k.tolist()))
wf_full = np.diff(wf_full, prepend=0)
qdeconv2 = deconv(wf_full, fr0)
print(np.sum(wf_k_trunc), np.sum(wf_full), np.sum(fr0_k), np.sum(qdeconv2), np.sum(wf))

print(qdeconv2.shape)
print(np.sum(qdeconv2), np.sum(q), np.sum(deconvq))
print(np.argmax(qdeconv2)*kticks, np.argmax(q), np.argmax(deconvq))
print(np.std(qdeconv2), np.std(q), np.std(deconvq))

print(len(qdeconv2), 'length')
plt.plot(np.arange(len(qdeconv2)), qdeconv2, label='interpolation from (meas + template)')
# plt.plot(deconvq, label='qhat')
plt.plot(q, label='true')
plt.legend()
plt.xlabel('Time tick[50ns]')

In [ ]:
plt.semilogy(np.fft.fftfreq(len(wf_full_k), d=kticks*0.05), np.abs(np.fft.fft(wf_full_k))**2, 'o', label='(mea+temp)^2')
plt.semilogy(np.fft.fftfreq(len(fr0_k), d=kticks*0.05), np.abs(np.fft.fft(fr0_k))**2, 'o', label='|FR integrate every k|^2')
plt.title('True charge @ 24ke and threshold @ 5ke')

plt.legend()
plt.xlabel('Frequency [MHz]')

In [ ]:
def rescale(wf_fft):
    return wf_fft/wf_fft[0]

plt.plot(np.fft.fftfreq(len(wf), d=1*0.05), rescale(np.abs(np.fft.fft(wf))**2), 'o', label='|true waveform|^2')
plt.plot(np.fft.fftfreq(len(fr0), d=1*0.05), rescale(np.abs(np.fft.fft(fr0))**2), 'o', label='|FR|^2 true')
plt.plot(np.fft.fftfreq(len(wf_full_k), d=kticks*0.05), rescale(np.abs(np.fft.fft(wf_full_k))**2), 'o', label='|measured+temp|^2')
plt.plot(np.fft.fftfreq(len(fr0_k), d=kticks*0.05), rescale(np.abs(np.fft.fft(fr0_k))**2), 'o', label='|FR integrate every k|^2')

# plt.axhline(1, label='noise level')
plt.title('Frequency spectra nomralized to a common height')
plt.legend()
plt.xlabel('Frequency [MHz]')

In [ ]:
def rescale(wf_fft):
    return wf_fft/wf_fft[0]

plt.semilogy(np.fft.fftfreq(len(wf), d=1*0.05), rescale(np.abs(np.fft.fft(wf))**2), 'o', ms=1, label='|true waveform| ^ 2')
plt.semilogy(np.fft.fftfreq(len(fr0), d=1*0.05), rescale(np.abs(np.fft.fft(fr0))**2), 'o', ms=1, label='|FR|^2 true')
plt.semilogy(np.fft.fftfreq(len(wf_full_k), d=kticks*0.05), rescale(np.abs(np.fft.fft(wf_full_k))**2), 'o', label='|measured + meas|^2')
plt.semilogy(np.fft.fftfreq(len(fr0_k), d=kticks*0.05), rescale(np.abs(np.fft.fft(fr0_k))**2), 'o', label='|FR integrate every k|^2')

plt.title('Frequency spectra nomralized to a common height')
plt.legend()
plt.xlabel('Frequency [MHz]')

In [ ]:
def rescale(wf_fft):
    return wf_fft/wf_fft[0]

plt.semilogy(np.fft.fftfreq(len(fr0), d=1*0.05), rescale(np.abs(np.fft.fft(fr0))**2), 'o', ms=1, label='|FR|^2 true')
plt.semilogy(np.fft.fftfreq(len(wf), d=1*0.05), rescale(np.abs(np.fft.fft(wf))**2), 'o', ms=1, label='true waveform')
plt.semilogy(np.fft.fftfreq(len(wf_full_k), d=kticks*0.05), rescale(np.abs(np.fft.fft(wf_full_k+np.random.normal(0, 1,  wf_full_k.shape)))**2), 'o', label='|meas+temp+gaus(0,1)|^2')
plt.semilogy(np.fft.fftfreq(len(fr0_k), d=kticks*0.05), rescale(np.abs(np.fft.fft(fr0_k))**2), 'o', label='|FR integrate every k|^2')
plt.title('Frequency spectra nomralized to the height at zero frequency')

# plt.axhline(1, label='noise level')
plt.legend()
plt.xlabel('Frequency [MHz]')

In [ ]:

plt.semilogy(np.fft.fftfreq(len(wf_full_k), d=kticks*0.05), rescale(np.abs(np.fft.fft(np.random.normal(0, 1, wf_full_k.shape)))**2), 'o', label='|meas+temp+gaus(0,1)|^2')
plt.legend()
plt.xlabel('Frequency [MHz]')

In [ ]:
def rescale(wf_fft):
    return wf_fft/wf_fft[0]

plt.plot(np.fft.fftfreq(len(wf_full_k), d=kticks*0.05), rescale(np.abs(np.fft.fft(wf_full_k+np.random.normal(0, 1,  wf_full_k.shape)))**2), 'o', label='temp+measured+gaus(0,1)')
plt.plot(np.fft.fftfreq(len(fr0_k), d=kticks*0.05), rescale(np.abs(np.fft.fft(fr0_k))**2), 'o', label='|FR integrate every k|^2')
plt.title('Frequency spectra nomralized to the height at zero frequency')

plt.legend()
plt.xlabel('Frequency [MHz]')

In [ ]:
def rescale(wf_fft):
    return wf_fft/wf_fft[0]

plt.plot(np.fft.fftfreq(len(wf_full_k), d=kticks*0.05), rescale(np.abs(np.fft.fft(wf_full_k))**2), 'o', label='|meas+temp|^2')
plt.plot(np.fft.fftfreq(len(fr0_k), d=kticks*0.05), rescale(np.abs(np.fft.fft(fr0_k))**2), 'o', label='|FR integrate every k|^2')
plt.title('Frequency spectra nomralized to the height at zero frequency')
# plt.axhline(1, label='noise level')
plt.legend()
plt.xlabel('Frequency [MHz]')

In [ ]:
noise = np.random.normal(0, 1,  wf_full_k.shape)
print(noise)
plt.semilogy(np.fft.fftfreq(len(wf_full_k), d=kticks*0.05), np.abs(np.fft.fft(wf_full_k+noise))**2, 'o', label='|meas+temp+gaus(0,1)|^2')
plt.semilogy(np.fft.fftfreq(len(wf_full_k), d=kticks*0.05), np.abs(np.fft.fft(wf_full_k))**2, 'o', label='|meas+temp|^2')
plt.semilogy(np.fft.fftfreq(len(wf_full_k), d=kticks*0.05), np.abs(np.fft.fft(noise))**2, 'o', label='|gaus(0,1)|^2')

plt.xlabel('Frequency [MHz]')
plt.legend()

In [ ]:

plt.plot(np.fft.fftfreq(len(wf_full_k), d=kticks*0.05), rescale(np.abs(np.fft.fft(wf_full_k))**2), 'o', label='|meas + temp|^2')
plt.plot(np.fft.fftfreq(len(fr0_k), d=kticks*0.05), rescale(np.abs(np.fft.fft(fr0_k))**2), 'o', label='|FR integrate every k|^2')
g2 =  gauss(np.arange(-0.2, 0.2+0.001, 0.1), 0, 0.1)
plt.plot(np.arange(-0.2, 0.2+0.001, 0.1), g2**2/np.max(g2)**2, 'o-', label='|Gaus(0, 0.1)|^2')
plt.xlabel('Frequnecy [MHz]')
plt.title('Frequency spectra nomralized to the height at zero frequency')

plt.legend()